<h2 align="center">点击下列图标在线运行HanLP</h2>
<div align="center">
	<a href="https://colab.research.google.com/github/hankcs/HanLP/blob/doc-zh/plugins/hanlp_demo/hanlp_demo/zh/tok_mtl.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
	<a href="https://mybinder.org/v2/gh/hankcs/HanLP/doc-zh?filepath=plugins%2Fhanlp_demo%2Fhanlp_demo%2Fzh%2Ftok_mtl.ipynb" target="_blank"><img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/></a>
</div>

## 安装

无论是Windows、Linux还是macOS，HanLP的安装只需一句话搞定：

In [ ]:
!pip install hanlp -U

## 加载模型
HanLP的工作流程是先加载模型，模型的标示符存储在`hanlp.pretrained`这个包中，按照NLP任务归类。

In [2]:
import hanlp
hanlp.pretrained.mtl.ALL # MTL多任务，具体任务见模型名称，语种见名称最后一个字段或相应语料库

{'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_base_20210111_124519.zip',
 'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_small_20210111_124159.zip',
 'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ERNIE_GRAM_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_ernie_gram_base_aug_20210904_145403.zip',
 'NPCMJ_UD_KYOTO_TOK_POS_CON_BERT_BASE_CHAR_JA': 'https://file.hankcs.com/hanlp/mtl/npcmj_ud_kyoto_tok_pos_ner_dep_con_srl_bert_base_char_ja_20210914_133742.zip',
 'OPEN_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH': 'https://file.hankcs.com/hanlp/mtl/open_tok_pos_ner_srl_dep_sdp_con_electra_base_20201223_201906.zip',
 'OPEN_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/mtl/open_tok_pos_ner_srl_dep_sdp_con_electra_small_20201223_035557.zip',
 'UD_ONTONOTES_TOK_POS_LEM_FEA_NER_SRL_DEP

调用`hanlp.load`进行加载，模型会自动下载到本地缓存。自然语言处理分为许多任务，分词只是最初级的一个。与其每个任务单独创建一个模型，不如利用HanLP的联合模型一次性完成多个任务：

In [3]:
HanLP = hanlp.load(hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH)

## 分词
任务越少，速度越快。如指定仅执行分词，默认细粒度：

In [4]:
HanLP('阿婆主来到北京立方庭参观自然语义科技公司。', tasks='tok').pretty_print()

执行粗颗粒度分词：

In [5]:
HanLP('阿婆主来到北京立方庭参观自然语义科技公司。', tasks='tok/coarse').pretty_print()

同时执行细粒度和粗粒度分词：

In [6]:
HanLP('阿婆主来到北京立方庭参观自然语义科技公司。', tasks='tok*')

{'tok/coarse': ['阿婆主', '来到', '北京立方庭', '参观', '自然语义科技公司', '。'],
 'tok/fine': ['阿婆主', '来到', '北京', '立方庭', '参观', '自然', '语义', '科技', '公司', '。']}

`coarse`为粗分，`fine`为细分。

#### 注意
Native API的输入单位限定为句子，需使用[多语种分句模型](https://github.com/hankcs/HanLP/blob/master/plugins/hanlp_demo/hanlp_demo/sent_split.py)或[基于规则的分句函数](https://github.com/hankcs/HanLP/blob/master/hanlp/utils/rules.py#L19)先行分句。RESTful同时支持全文、句子、已分词的句子。除此之外，RESTful和native两种API的语义设计完全一致，用户可以无缝互换。

## 自定义词典
自定义词典为分词任务的成员变量，要操作自定义词典，先获取分词任务，以细分标准为例：

In [7]:
tok = HanLP['tok/fine']
tok

不挂词典：

In [8]:
tok.dict_force = tok.dict_combine = None
HanLP("商品和服务项目")["tok/fine"]

['商品', '和', '服务', '项目']

强制模式（慎用，详见[《自然语言处理入门》](http://nlp.hankcs.com/book.php)第二章）：

In [9]:
tok.dict_force = {'和服', '服务项目'}
HanLP("商品和服务项目")["tok/fine"]

['商品', '和服', '务', '项目']

强制校正:

In [10]:
tok.dict_force = {'和服务': ['和', '服务']}
HanLP("正向匹配商品和服务、任何和服务必按上述切分")["tok/fine"]

['正向', '匹配', '商品', '和', '服务', '、', '任何', '和', '服务', '必', '按', '上述', '切分']

合并模式:

In [11]:
tok.dict_force = None
tok.dict_combine = {'和服', '服务项目'}
HanLP("商品和服务项目")["tok/fine"]

['商品', '和', '服务项目']

需要算法基础才能理解，初学者可参考[《自然语言处理入门》](http://nlp.hankcs.com/book.php)。